In [3]:
#

In [4]:
using StaticArrays

In [5]:
@enum Color::Int empty blue red black white green yellow

In [6]:
Color

Enum Color:
empty = 0
blue = 1
red = 2
black = 3
white = 4
green = 5
yellow = 6

In [7]:
const color_letters = Dict("e" => empty, "b" => blue, "r" => red, "k" => black, "w" => white, "g" => green, "y" => yellow)

Dict{String,Color} with 7 entries:
  "g" => green
  "w" => white
  "e" => empty
  "k" => black
  "b" => blue
  "r" => red
  "y" => yellow

In [8]:
const npegs = 4

4

In [9]:
struct Guess
    pegs::SVector{npegs, Color} 
    #pegs::Vector{Color}
    #pegs::NTuple{npegs, Color}
end

In [10]:
#Guess(array::Array) = Guess(tuple(array...))

In [11]:
Guess() = Guess(fill(empty, npegs))

Guess

In [12]:
Guess(s::String) = Guess([color_letters[string(c)] for c in s])

Guess

In [13]:
Base.show(io::IO, g::Guess) = print(io, "[", join(string.(g.pegs), " "), "]")

In [14]:
const ncolors = length(instances(Color))

7

In [15]:
random_color() = Color(rand(0:ncolors-1))

random_color (generic function with 1 method)

In [16]:
random_color(n) = Color.(rand(0:ncolors-1, n))

random_color (generic function with 2 methods)

In [17]:
random_guess() = Guess(random_color(npegs))

random_guess (generic function with 1 method)

In [18]:
struct Answer
    blacks::Int
    whites::Int
end

In [19]:
all_black(a::Answer) = a.blacks == npegs

all_black (generic function with 1 method)

In [20]:
Base.show(io::IO, a::Answer) = print(io, "[blacks=$(a.blacks) whites=$(a.whites)]")

In [21]:
function compare(g1::Guess, g2::Guess)
    blacks, whites = 0, 0
    used1 = @MVector zeros(Bool, npegs)
    used2 = @MVector zeros(Bool, npegs)
    for i = 1:npegs
        if g1.pegs[i] == g2.pegs[i]
            blacks += 1
            used1[i], used2[i] = true, true
        end
    end
    for i = 1:npegs
        for j = 1:npegs
            if i != j && !used1[i] && !used2[j] && g1.pegs[i] == g2.pegs[j]
                whites += 1
            used1[i], used2[j] = true, true
            end
        end
    end
    return Answer(blacks, whites)
end

compare (generic function with 1 method)

In [23]:
let g1 = random_guess(), g2 = random_guess()
    println("$g1 $g2 ", compare(g1, g2))
    println(g1 == g2)
end

[red blue red green] [red white empty empty] [blacks=1 whites=0]
false


In [24]:
struct Fact
    guess::Guess
    answer::Answer
end

In [25]:
allows(f::Fact, g::Guess) = f.answer == compare(f.guess, g)

allows (generic function with 1 method)

In [26]:
allows(fs::Vector{Fact}, g::Guess) = all(allows(f, g) for f in fs)

allows (generic function with 2 methods)

In [27]:
facts = [Fact(Guess("rwge"), Answer(3, 0)), Fact(Guess("ykbb"), Answer(0, 1))]
allows(facts, Guess("rwgy"))

true

In [28]:
struct FrequencyList
    fl::Dict{Answer, Int}
end

In [29]:
FrequencyList() = FrequencyList(Dict{Answer, Int}())

FrequencyList

In [30]:
count_allblacks(fl::FrequencyList) = fl.fl[Answer(npegs, 0)]

count_allblacks (generic function with 1 method)

In [31]:
function info_value(fl::FrequencyList)
    r = 0.0
    ntot = 0
    for v in values(fl.fl)
        ntot += 1
        if v != 0 && v != 1
            r -= v * log(v)
        end
    end
    if ntot > 0
        return (r/ntot + log(ntot)) / log(2.0)
    else
        return 0.0
    end
end

info_value (generic function with 1 method)

In [32]:
info_value(FrequencyList(Dict(Answer(3,0)=>3, Answer(2,2)=>20)))

-44.59672469995536

In [33]:
using StatsBase

In [34]:
function calculate_all_guesses!(result, prev_colors, npegs=npegs)
    if length(prev_colors) == npegs
        push!(result, Guess(prev_colors))
    else
        for color in instances(Color)
            calculate_all_guesses!(result, [prev_colors; color], npegs)
        end
    end
end

calculate_all_guesses! (generic function with 2 methods)

In [35]:
function calculate_all_guesses(npegs=npegs)
    result = Guess[]
    calculate_all_guesses!(result, Color[], npegs)
    return result
end

calculate_all_guesses (generic function with 2 methods)

In [36]:
const all_guesses = calculate_all_guesses();

In [37]:
@enum GameState notfound found impossible

In [38]:
using StatsBase

In [39]:
const fixed_first_guess = Guess([yellow, blue, red, black])

[yellow blue red black]

In [40]:
const random_first_guess = false

false

In [41]:
function first_guess()
    if !random_first_guess
        return fixed_first_guess
    end
    replace = ncolors < npegs
    return Guess(sample(collect(instances(Color)), npegs, replace=replace))
end

first_guess (generic function with 1 method)

In [42]:
Color

Enum Color:
empty = 0
blue = 1
red = 2
black = 3
white = 4
green = 5
yellow = 6

In [43]:
function make_guess(facts::Vector{Fact})
    if length(facts) == 0
        return first_guess(), notfound, ncolors^npegs
    end
    possible_solutions = [guess for guess in all_guesses if allows(facts, guess)]
    if length(possible_solutions) == 1
        return possible_solutions[1], found, 1
    elseif length(possible_solutions) == 0
        return Guess(), impossible, 0
    end
    info_values = zeros(length(all_guesses)) # fill(-Inf, length(all_guesses))
    for i = 1:length(all_guesses)
        guess = all_guesses[i]
        fl = FrequencyList()
        for solution in possible_solutions
            answer = compare(all_guesses[i], solution)
            fl.fl[answer] = get(fl.fl, answer, 0) + 1
        end
        #@time fl = FrequencyList(countmap(compare(guess, solution) for solution in possible_solutions))
        #@show fl
        info_values[i] = info_value(fl)
        #@show info_values[i]
    end
    #@show sort(-info_values)[1:4]
    max_info_value = maximum(info_values)
    idx_best_guesses = findall((x->x==max_info_value), info_values)
    for idx in idx_best_guesses
        if allows(facts, all_guesses[idx])
            return all_guesses[idx], notfound, length(possible_solutions)
        end
    end
    return all_guesses[idx_best_guesses[1]], notfound, length(possible_solutions)
end

make_guess (generic function with 1 method)

In [47]:
function play(ask)
    facts = Vector{Fact}()
    guess, state = Guess(), notfound
    counter = 0
    while state == notfound
        @time guess, state, possibilities = make_guess(facts)
        println("$possibilities possible solutions left.")
        if state == impossible
            break
        end
        answer = ask(guess)
        if answer == nothing
            println("quitting")
            return
        elseif all_black(answer)
            state = found
        else
            push!(facts, Fact(guess, answer))
        end
        counter += 1
    end
    if state == found
        println("Found solution: $guess in $counter steps.")
    else
        println("No solution possible!")
    end
end

function play_automatically(solution) 
    function ask(guess) 
        answer = compare(guess, solution)
        @show guess answer
        return answer
    end
    play(ask)
end

play_automatically (generic function with 1 method)

In [48]:
# warm up for JIT
play_automatically(Guess([red,white,green,empty]))

  0.000002 seconds
2401 possible solutions left.
guess = [yellow blue red black]
answer = [blacks=0 whites=1]
  0.089869 seconds (11.53 k allocations: 2.924 MiB)
444 possible solutions left.
guess = [empty empty blue white]
answer = [blacks=0 whites=2]
  0.021892 seconds (10.05 k allocations: 1.976 MiB)
64 possible solutions left.
guess = [blue white green green]
answer = [blacks=2 whites=0]
  0.011510 seconds (9.62 k allocations: 1.712 MiB, 78.42% gc time)
5 possible solutions left.
guess = [red white empty green]
answer = [blacks=2 whites=2]
  0.000157 seconds (3 allocations: 304 bytes)
1 possible solutions left.
guess = [red white green empty]
answer = [blacks=4 whites=0]
Found solution: [red white green empty] in 5 steps.


In [57]:
@time play_automatically(Guess([red,white,green,empty]))

  0.000000 seconds
2401 possible solutions left.
guess = [yellow blue red black]
answer = [blacks=0 whites=1]
  0.116376 seconds (11.53 k allocations: 2.924 MiB)
444 possible solutions left.
guess = [empty empty blue white]
answer = [blacks=0 whites=2]
  0.016157 seconds (10.05 k allocations: 1.976 MiB)
64 possible solutions left.
guess = [blue white green green]
answer = [blacks=2 whites=0]
  0.003597 seconds (9.62 k allocations: 1.712 MiB)
5 possible solutions left.
guess = [red white empty green]
answer = [blacks=2 whites=2]
  0.000150 seconds (3 allocations: 304 bytes)
1 possible solutions left.
guess = [red white green empty]
answer = [blacks=4 whites=0]
Found solution: [red white green empty] in 5 steps.
  0.142632 seconds (32.33 k allocations: 6.663 MiB)


In [50]:
function play_manually()
    println("Please think of a color for $npegs pegs.")
    println("Possible colors are $(instances(Color)).")
    println("For each question, respond with the number of black pegs and white pegs, separated by a space.")
    println("Enter 'q' to quit.")
    function ask(guess)
        println()
        println("My guess is $guess.")
        println("Black pegs, white pegs?")
        response = readline()
        if startswith(response, "q")
            return nothing
        else
            blacks, whites = parse.(Int, split(response))
            return Answer(blacks, whites)
        end
    end
    play(ask)
end

play_manually (generic function with 1 method)

In [58]:
play_manually()

Please think of a color for 4 pegs.
Possible colors are (empty, blue, red, black, white, green, yellow).
For each question, respond with the number of black pegs and white pegs, separated by a space.
Enter 'q' to quit.
  0.000001 seconds
2401 possible solutions left.

My guess is [yellow blue red black].
Black pegs, white pegs?


stdin>  0 1


  0.114996 seconds (11.53 k allocations: 2.924 MiB)
444 possible solutions left.

My guess is [empty empty blue white].
Black pegs, white pegs?


stdin>  0 2


  0.025993 seconds (10.05 k allocations: 1.976 MiB)
64 possible solutions left.

My guess is [blue white green green].
Black pegs, white pegs?


...

  0.004836 seconds (9.62 k allocations: 1.705 MiB)
9 possible solutions left.

My guess is [white green empty red].
Black pegs, white pegs?


stdin>  0 4


  0.000165 seconds (3 allocations: 304 bytes)
1 possible solutions left.

My guess is [green red white empty].
Black pegs, white pegs?


stdin>  4 0


Found solution: [green red white empty] in 5 steps.


In [44]:
using BenchmarkTools

In [45]:
let g1 = random_guess(), g2 = random_guess()
    @benchmark answer = compare($g1, $g2)
end

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     14.528 ns (0.00% GC)
  median time:      17.134 ns (0.00% GC)
  mean time:        22.373 ns (0.00% GC)
  maximum time:     3.030 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     998

In [46]:
let g1 = random_guess(), g2 = random_guess()
    answer = compare(g1, g2)
    fl = FrequencyList()
    @benchmark $fl.fl[$answer] = get($fl.fl, $answer, 0) + 1
end

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     42.539 ns (0.00% GC)
  median time:      44.154 ns (0.00% GC)
  mean time:        49.764 ns (0.00% GC)
  maximum time:     593.246 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     992